In [ ]:
!pip install langdetect

import pandas as pd
import numpy as np
import re
from langdetect import detect, DetectorFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

DetectorFactory.seed = 0

# ==============================================================================
# 1. CẤU HÌNH BỘ LỌC
# ==============================================================================
MIN_CHARS = 70      # Độ dài tối thiểu
MIN_WORDS = 15      # Số từ tối thiểu
DUPLICATE_THRESHOLD = 0.85 # Ngưỡng trùng lặp

# Danh sách từ khóa "RÁC" cần loại bỏ ngay lập tức
SPAM_KEYWORDS = [
    "subscribe", "đăng ký kênh", "bấm chuông", "follow kênh",
    "ghiền mì gõ", "ghien mi go", "ghienmigo", "faptv", "Ghiền Mì Gõ"
    "copyright", "bản quyền", "link trong bio",
    "http://", "https://", "www."
]

# ==============================================================================
# 2. CÁC HÀM XỬ LÝ
# ==============================================================================

def clean_text_basic(text):
    """Làm sạch sơ bộ ký tự lạ"""
    if not isinstance(text, str): return ""
    # Xóa ký tự lạ, giữ lại dấu câu
    text = re.sub(r'[^\w\s.,!?;:"\'\-%/()]', ' ', text)
    # Xóa khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def is_spam_content(text):
    """
    Kiểm tra xem text có chứa từ khóa rác hoặc là link URL không.
    Trả về TRUE nếu là SPAM.
    """
    text_lower = text.lower()

    # 1. Check từ khóa cấm
    for keyword in SPAM_KEYWORDS:
        if keyword in text_lower:
            return True

    # 2. Check nếu toàn bộ text nhìn giống URL
    if re.match(r'^[a-zA-Z0-9\.\-_]+$', text): # Kiểu "vtv.vn" hoặc "tiktok.com"
        return True

    return False

def is_valid_quality(text):
    """Kiểm tra độ dài và chất lượng nội dung"""
    # 1. Độ dài
    if len(text) < MIN_CHARS: return False
    words = text.split()
    if len(words) < MIN_WORDS: return False

    # 2. Tỷ lệ số (tránh dòng toàn số liệu vô nghĩa)
    digit_count = sum(c.isdigit() for c in text)
    if digit_count / len(text) > 0.5: return False

    return True

def is_vietnamese(text):
    """Kiểm tra ngôn ngữ"""
    try:
        if len(text.split()) < 3: return False
        return detect(text) == 'vi'
    except:
        return False

# ==============================================================================
# 3. HÀM CHÍNH: CLEAN FILE REAL
# ==============================================================================

def clean_real_dataset_specifically():
    print("🚀 BẮT ĐẦU LÀM SẠCH DATASET REAL")

    # --- ĐƯỜNG DẪN FILE (Sửa lại tên file của bạn) ---
    # input_csv = "/content/drive/MyDrive/TikTok_Project/tiktok_videos_all_keywords_real_done.csv"
    # output_csv = "/content/drive/MyDrive/TikTok_Project/dataset_real_final_clean.csv"

    input_csv = "/content/drive/MyDrive/TikTok_Project/dataset_fake.csv"
    output_csv = "/content/drive/MyDrive/TikTok_Project/dataset_fake_final_clean.csv"

    try:
        df = pd.read_csv(input_csv)
        print(f"📊 Tổng số dòng ban đầu: {len(df)}")
    except Exception as e:
        print(f"❌ Lỗi đọc file: {e}")
        return

    # 1. Xóa dòng trống
    df = df.dropna(subset=['text'])
    df['text'] = df['text'].astype(str).apply(clean_text_basic)

    # 2. LỌC SPAM (Subscribe, Mì Gõ, URL...)
    print("🗑️ Đang quét và xóa các câu Spam/Intro rác...")
    # Tạo mask những dòng KHÔNG phải spam
    df['is_clean'] = ~df['text'].apply(is_spam_content)
    df = df[df['is_clean'] == True].drop(columns=['is_clean'])
    print(f"   -> Sau khi lọc Spam: còn {len(df)} dòng")

    # 3. LỌC CHẤT LƯỢNG (Độ dài)
    print(f"✂️ Đang lọc câu ngắn (<{MIN_WORDS} từ)...")
    df['is_valid'] = df['text'].apply(is_valid_quality)
    df = df[df['is_valid'] == True].drop(columns=['is_valid'])
    print(f"   -> Sau khi lọc ngắn: còn {len(df)} dòng")

    # 4. LỌC TIẾNG VIỆT
    print("🌍 Đang kiểm tra ngôn ngữ...")
    df['is_vi'] = df['text'].apply(is_vietnamese)
    df = df[df['is_vi'] == True].drop(columns=['is_vi'])

    # 5. XÓA TRÙNG LẶP (FUZZY)
    print("🤖 Đang xóa trùng lặp nội dung (>85%)...")
    df = df.drop_duplicates(subset=['text'], keep='first').reset_index(drop=True)

    if len(df) > 1:
        tfidf = TfidfVectorizer(min_df=1).fit_transform(df['text'])
        cosine_sim = cosine_similarity(tfidf, tfidf)
        to_drop = set()
        for i in range(len(df)):
            if i in to_drop: continue
            for j in range(i + 1, len(df)):
                if j in to_drop: continue
                if cosine_sim[i, j] > DUPLICATE_THRESHOLD:
                    to_drop.add(j)
        df_final = df.drop(index=list(to_drop))
    else:
        df_final = df

    # 6. CHUẨN HÓA CATEGORY
    df_final['category'] = 'real'

    # --- LƯU KẾT QUẢ ---
    print("\n" + "="*40)
    print(f"✅ HOÀN TẤT FILE REAL!")
    print(f"👉 Còn lại: {len(df_final)} dòng chất lượng cao.")
    print(f"👉 File lưu tại: {output_csv}")
    print("="*40)

    df_final.to_csv(output_csv, index=False, encoding='utf-8-sig')

# Chạy hàm
clean_real_dataset_specifically()

🚀 BẮT ĐẦU LÀM SẠCH DATASET REAL
📊 Tổng số dòng ban đầu: 1590
🗑️ Đang quét và xóa các câu Spam/Intro rác...
   -> Sau khi lọc Spam: còn 1415 dòng
✂️ Đang lọc câu ngắn (<15 từ)...
   -> Sau khi lọc ngắn: còn 1415 dòng
🌍 Đang kiểm tra ngôn ngữ...
🤖 Đang xóa trùng lặp nội dung (>85%)...

✅ HOÀN TẤT FILE REAL!
👉 Còn lại: 1415 dòng chất lượng cao.
👉 File lưu tại: /content/drive/MyDrive/TikTok_Project/dataset_fake_final_clean.csv


In [ ]:
import pandas as pd
import os

def merge_fake_real_final():
    print("🚀 BẮT ĐẦU GỘP DATASET FAKE VÀ REAL...")

    # ==============================================================================
    # 1. CẤU HÌNH ĐƯỜNG DẪN (Bạn thay đúng tên file của bạn vào đây)
    # ==============================================================================

    # File Fake (đã có đủ metadata và text sạch)
    path_fake = "/content/drive/MyDrive/TikTok_Project/dataset_fake_final_clean.csv"

    # File Real (đã lọc sạch spam, intro rác)
    path_real = "/content/drive/MyDrive/TikTok_Project/dataset_real_final_clean.csv"

    # File kết quả cuối cùng (Sẽ dùng file này để Train AI)
    path_final = "/content/drive/MyDrive/TikTok_Project/dataset_TIKTOK_FAKE_NEWS_FINAL.csv"

    # ==============================================================================
    # 2. ĐỌC DỮ LIỆU
    # ==============================================================================
    try:
        print(f"📂 Đang đọc Fake: {path_fake}")
        df_fake = pd.read_csv(path_fake)
        # Đảm bảo nhãn đúng
        df_fake['category'] = 'fake'
        print(f"   -> Số lượng Fake: {len(df_fake)} dòng")

        print(f"📂 Đang đọc Real: {path_real}")
        df_real = pd.read_csv(path_real)
        # Đảm bảo nhãn đúng
        df_real['category'] = 'real'
        print(f"   -> Số lượng Real: {len(df_real)} dòng")

    except Exception as e:
        print(f"❌ Lỗi đọc file: {e}")
        return

    # ==============================================================================
    # 3. GỘP (CONCAT)
    # ==============================================================================
    print("🔗 Đang ghép 2 file lại với nhau...")

    # ignore_index=True để reset lại số thứ tự từ 0 đến hết
    df_merged = pd.concat([df_fake, df_real], ignore_index=True)

    print(f"📊 Tổng số dữ liệu sau khi gộp: {len(df_merged)} dòng")

    # ==============================================================================
    # 4. SẮP XẾP CỘT (REORDER COLUMNS)
    # ==============================================================================
    # Sắp xếp lại thứ tự cột cho chuẩn, đưa các thông tin quan trọng lên đầu
    # Các cột metadata ít quan trọng hơn thì để ra sau

    desired_order = [
        'url', 'text', 'category',          # 3 cột quan trọng nhất cho Model
        'desc', 'keyword', 'createTime',    # Thông tin nội dung
        'shareCount', 'commentCount', 'playCount', 'diggCount', 'collectCount', # Tương tác
        'author_nickname', 'author_unique_id', 'author_id', 'video_id', 'thumnail_url' # Metadata tác giả
    ]

    # Chỉ lấy những cột có trong dữ liệu thực tế (đề phòng thiếu cột nào đó)
    existing_cols = [c for c in desired_order if c in df_merged.columns]

    # Nếu còn cột nào lạ chưa có trong danh sách trên thì thêm vào đuôi
    remaining_cols = [c for c in df_merged.columns if c not in existing_cols]

    final_cols = existing_cols + remaining_cols

    df_merged = df_merged[final_cols]

    # ==============================================================================
    # 5. XÁO TRỘN DỮ LIỆU (SHUFFLE) - TÙY CHỌN
    # ==============================================================================
    # Để khi mở file lên không bị kiểu: 1000 dòng đầu toàn Fake, 1000 dòng sau toàn Real
    # Giúp cái nhìn khách quan hơn
    print("🔀 Đang xáo trộn ngẫu nhiên thứ tự dòng...")
    df_merged = df_merged.sample(frac=1, random_state=42).reset_index(drop=True)

    # ==============================================================================
    # 6. LƯU KẾT QUẢ
    # ==============================================================================
    df_merged.to_csv(path_final, index=False, encoding='utf-8-sig')

    print("\n" + "="*50)
    print("🎉 CHÚC MỪNG! BẠN ĐÃ CÓ DATASET HOÀN CHỈNH!")
    print(f"👉 Đường dẫn: {path_final}")
    print(f"👉 Tổng số dòng: {len(df_merged)}")
    print("-" * 30)
    print("Phân bố nhãn:")
    print(df_merged['category'].value_counts())
    print("="*50)

# Chạy hàm
merge_fake_real_final()

🚀 BẮT ĐẦU GỘP DATASET FAKE VÀ REAL...
📂 Đang đọc Fake: /content/drive/MyDrive/TikTok_Project/dataset_fake_final_clean.csv
   -> Số lượng Fake: 1415 dòng
📂 Đang đọc Real: /content/drive/MyDrive/TikTok_Project/dataset_real_final_clean.csv
   -> Số lượng Real: 1069 dòng
🔗 Đang ghép 2 file lại với nhau...
📊 Tổng số dữ liệu sau khi gộp: 2484 dòng
🔀 Đang xáo trộn ngẫu nhiên thứ tự dòng...

🎉 CHÚC MỪNG! BẠN ĐÃ CÓ DATASET HOÀN CHỈNH!
👉 Đường dẫn: /content/drive/MyDrive/TikTok_Project/dataset_TIKTOK_FAKE_NEWS_FINAL.csv
👉 Tổng số dòng: 2484
------------------------------
Phân bố nhãn:
category
fake    1415
real    1069
Name: count, dtype: int64
